In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord,EarthLocation
import astropy.units as u
from astropy.units import imperial
from astropy.time import Time
from astropy.table import Table
import astropy.constants as const
import numpy as np
imperial.enable()
import glob
import os

In [ ]:
(13658.14 * imperial.ft).to(u.m)

In [ ]:
latSub, lonSub = 19. + 49./60. + 31.8142/3600.,-(155.+28./60.+33.66719/3600.)
latSub, lonSub

In [ ]:
subaru = EarthLocation.from_geodetic(lat=latSub * u.deg,lon=lonSub * u.deg,
                                     height=(13658.14 * imperial.ft).to(u.m))
#s1 = EarthLocation.of_site('Subaru') from https://www.ifa.hawaii.edu/mko/coordinates.shtml

In [ ]:
## Gaia DR2 position
c1 = SkyCoord(ra=290.9662101686864 * u.deg,dec=51.5047656451624 * u.deg,distance=618.6 * u.pc)
c1.to_string('hmsdms',sep=' ')

In [ ]:
list1 = glob.glob('../fits_data/proc/out_HDSA000940*.fits')
list2 = glob.glob('../hirano/kic1255_20150827/w*.fits')
list3 = glob.glob('../hirano/kic1255_20150828/w*.fits')
fileNames, allDates = [], []
for oneList in [list1,list2,list3]:
    for oneFile in oneList:
        head = fits.getheader(oneFile)
        #if head['WAVELEN'] == 583.74: ## only include blue side
        fileNames.append(os.path.basename(oneFile))
        allDates.append(head['DATE-OBS']+'T'+head['UT-STR'])

In [ ]:
tArray = Time(allDates)

In [ ]:
barycorr = c1.radial_velocity_correction(obstime=tArray, location=subaru)


In [ ]:
np.round(5160. * u.AA * barycorr/const.c,4)

In [ ]:
t = Table()
t['File Name'] = fileNames
t['Time'] = tArray
t['JD'] = tArray.jd
t['RV (km/s)'] = np.round(barycorr.to(u.km/u.s),4)
t

In [ ]:
t.write('output/barycent_corrections.csv',overwrite=True)